In [11]:
# Convert columns in .csv file to .kml file for use in Google Earth

import pandas as pd
from xml.etree.ElementTree import Element, SubElement, tostring, ElementTree

def create_kml(csv_file, kml_file):
    df = pd.read_csv(csv_file)
    
    # Create KML root
    kml = Element("kml", xmlns="http://www.opengis.net/kml/2.2")
    document = SubElement(kml, "Document")
    
    for _, row in df.iterrows():
        placemark = SubElement(document, "Placemark")
        name = SubElement(placemark, "name")
        name.text = str(row["name"])  # Ensure name is string
        
        point = SubElement(placemark, "Point")
        coordinates = SubElement(point, "coordinates")
        coordinates.text = f"{row['longitude']},{row['latitude']},0"
    
    # Write to KML file
    tree = ElementTree(kml)
    tree.write(kml_file, encoding="utf-8", xml_declaration=True)
    print(f"KML file saved as {kml_file}")

import pandas as pd
from xml.etree.ElementTree import Element, SubElement, ElementTree

def elevation_to_kml_color(elev_m, min_elev, max_elev):
    """
    Converts elevation to a KML-compatible color gradient from yellow to red.
    Colors are in KML's AABBGGRR format.
    """
    if pd.isna(elev_m):  # Default gray if no elevation
        return "ff888888"

    # Normalize elevation (0 to 1)
    norm = (elev_m - min_elev) / (max_elev - min_elev) if max_elev > min_elev else 0.5

    # Start with yellow (R=255, G=255), go to red (R=255, G=0)
    red = 255
    green = int(255 * (1 - norm))  # from 255 → 0
    blue = 0

    return f"ff{blue:02x}{green:02x}{red:02x}"  # KML format: AABBGGRR

def elevation_to_kml_color(elev_m, min_elev, max_elev):
    """
    Converts elevation to a KML-compatible color gradient (Blue → Green → Red).
    Colors are in KML's AABBGGRR format.
    """
    if pd.isna(elev_m):  # Default gray if no elevation
        return "ff888888"

    # Normalize elevation (0 to 1)
    norm = (elev_m - min_elev) / (max_elev - min_elev) if max_elev > min_elev else 0.5

    # Define color gradient from blue (low) → green → red (high)
    if norm < 0.5:
        blue = int(255 * (1 - 2 * norm))
        green = int(255 * (2 * norm))
        red = 0
    else:
        blue = 0
        green = int(255 * (2 * (1 - norm)))
        red = int(255 * (2 * (norm - 0.5)))

    return f"ff{blue:02x}{green:02x}{red:02x}"  # KML format (AABBGGRR)

def create_HWM_kml(csv_file, kml_file):
    df = pd.read_csv(csv_file)

    # Check if required columns exist
    required_columns = {"site_no", "longitude_dd", "latitude_dd", "elev_ft"}
    if not required_columns.issubset(df.columns):
        raise KeyError(f"Missing required columns: {required_columns - set(df.columns)}")

    # Convert elevation from feet to meters
    df["elev_m"] = df["elev_ft"] * 0.3048

    # Determine min/max elevation for color scaling
    min_elev, max_elev = df["elev_m"].min(), df["elev_m"].max()

    # Create KML root
    kml = Element("kml", xmlns="http://www.opengis.net/kml/2.2")
    document = SubElement(kml, "Document")

    for _, row in df.iterrows():
        placemark = SubElement(document, "Placemark")

        # Use site number and elevation in name
        name = SubElement(placemark, "name")
        name.text = f"{row['site_no']} (Elev: {row['elev_m']:.2f}m)"

        # Define colored style based on elevation
        style = SubElement(placemark, "Style")
        icon_style = SubElement(style, "IconStyle")
        color = SubElement(icon_style, "color")
        color.text = elevation_to_kml_color(row["elev_m"], min_elev, max_elev)

        # Set balloon icon instead of pushpin
        icon = SubElement(icon_style, "Icon")
        icon_href = SubElement(icon, "href")
        icon_href.text = "http://maps.google.com/mapfiles/kml/paddle/blu-blank.png"  # Balloon icon

        # Balloon-style description
        balloon_style = SubElement(style, "BalloonStyle")
        text = SubElement(balloon_style, "text")
        text.text = f"<![CDATA[<b>Site:</b> {row['site_no']}<br><b>Elevation:</b> {row['elev_m']:.2f}m]]>"

        # Define coordinates
        point = SubElement(placemark, "Point")
        coordinates = SubElement(point, "coordinates")
        coordinates.text = f"{row['longitude_dd']},{row['latitude_dd']},0"

    # Write to KML file
    tree = ElementTree(kml)
    tree.write(kml_file, encoding="utf-8", xml_declaration=True)
    print(f"KML file saved as {kml_file}")


In [13]:
# Station locations with regular pushpins  
data_dir = 'F:/crs/proj/2025_NOPP_comparison/helene_water_level/'
csv_file = "all_sites_for_KML.csv"  # Update with the actual path
kml_file = "stations_v2.kml"
create_kml(data_dir+csv_file, data_dir+kml_file)

KML file saved as F:/crs/proj/2025_NOPP_comparison/helene_water_level/stations_v2.kml


In [12]:
# Wave stations with colored balloons
hwm_csv_file = "FilteredHWMs_Helene.csv"
hwm_kml_file = "hwm.kml"
create_HWM_kml(data_dir+hwm_csv_file, data_dir+hwm_kml_file)

KML file saved as F:/crs/proj/2025_NOPP_comparison/helene_water_level/hwm.kml
